In [7]:
!pip install unicode

You should consider upgrading via the 'c:\users\catae\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [10]:
!pip install unidecode

You should consider upgrading via the 'c:\users\catae\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [151]:
import pandas as pd
import numpy as np
import unidecode

## Functions

In [152]:
def modify_string(a):
    return unidecode.unidecode(a).strip().lower()

In [153]:
def modify_int(a):
    if isinstance(a, int):
        return a
    else:
        return np.nan
    

In [154]:
def store_df(df, filename):
    df.to_csv(filename, sep=',', encoding='utf-8')

## Limpieza

### Candidaturas 2021

In [309]:
df_2021 = pd.read_csv("Candidaturas_2021.csv", sep=";")
df_2021.head()

,ID Eleccion,ID Region,Region,Territorio,Candidato(a),Nombre,Paterno,Materno,Sexo,Edad,Rango,ID Partido,Partido
0,ALCALDES,1.0,DE TARAPACA,ALTO HOSPICIO,CAMILA ARCE FAJARDO,CAMILA,ARCE,FAJARDO,FEMENINO,38.0,35-39,IND,INDEPENDIENTES
1,ALCALDES,1.0,DE TARAPACA,ALTO HOSPICIO,GERMAN ANTONIO QUIROZ CANCINO,GERMAN ANTONIO,QUIROZ,CANCINO,MASCULINO,68.0,65-69,IND,INDEPENDIENTES
2,ALCALDES,1.0,DE TARAPACA,ALTO HOSPICIO,JORGE ZAVALA VALENZUELA,JORGE,ZAVALA,VALENZUELA,MASCULINO,42.0,40-44,IND,INDEPENDIENTES
3,ALCALDES,1.0,DE TARAPACA,ALTO HOSPICIO,PATRICIO FERREIRA RIVERA,PATRICIO,FERREIRA,RIVERA,MASCULINO,54.0,50-54,PDC,PARTIDO DEMOCRATA CRISTIANO
4,ALCALDES,1.0,DE TARAPACA,ALTO HOSPICIO,PAULA ANDREA ASTUDILLO DIAZ,PAULA ANDREA,ASTUDILLO,DIAZ,FEMENINO,33.0,30-34,IND,INDEPENDIENTES


In [310]:
#cambia orden de nombre pa seguir formato de la otra db
df_2021.dropna(inplace=True)
df_2021['Candidato(a)'] = df_2021[['Paterno', 'Materno', 'Nombre']].agg(' '.join, axis=1)

In [311]:
# ID de tabla se genera al poblar DB
new_2021 = pd.DataFrame(columns={
    'Ano': pd.Series(dtype=int),
    'ID Eleccion': pd.Series(dtype=str),
    'ID Region': pd.Series(dtype=float),
    'Region': pd.Series(dtype=str),
    'Territorio': pd.Series(dtype=str),
    'Candidato': pd.Series(dtype=str),
    'Sexo': pd.Series(dtype=str),
    'Edad': pd.Series(dtype=int),
    'ID Partido': pd.Series(dtype=str),
    'Partido': pd.Series(dtype=str)
})

def fill_candidaturas(new, old):
    new['ID Eleccion'] = old['ID Eleccion'].astype(str).apply(modify_string)
    new['ID Region'] = old['ID Region'].astype(float)
    new['Region'] = old['Region'].astype(str).apply(modify_string)
    new['Territorio'] = old['Territorio'].astype(str).apply(modify_string)
    new['Candidato'] = old['Candidato(a)'].astype(str).apply(modify_string)
    new['Sexo'] = old['Sexo'].astype(str).apply(modify_string)
    new['Edad'] = old['Edad'].astype(int).apply(modify_int)
    new['ID Partido'] = old['ID Partido'].astype(str).apply(modify_string)
    new['Partido'] = old['Partido'].astype(str).apply(modify_string)
    # elimino duplicados
    new = new.drop_duplicates()
    return new

#para generar los diccionarios partido id, region e id, etc.
def generate_dictionaries(col1, col2):
    dictionary = {}
    for i in range(len(col2)):
        element = col2.values[i]
        if element not in dictionary:
            dictionary[element] = col1.values[i]
    return dictionary

new_2021 = fill_candidaturas(new_2021, df_2021)
new_2021["Ano"] = 2021

new_2021.head()

,Ano,ID Eleccion,ID Region,Region,Territorio,Candidato,Sexo,Edad,ID Partido,Partido
0,2021,alcaldes,1.0,de tarapaca,alto hospicio,arce fajardo camila,femenino,38,ind,independientes
1,2021,alcaldes,1.0,de tarapaca,alto hospicio,quiroz cancino german antonio,masculino,68,ind,independientes
2,2021,alcaldes,1.0,de tarapaca,alto hospicio,zavala valenzuela jorge,masculino,42,ind,independientes
3,2021,alcaldes,1.0,de tarapaca,alto hospicio,ferreira rivera patricio,masculino,54,pdc,partido democrata cristiano
4,2021,alcaldes,1.0,de tarapaca,alto hospicio,astudillo diaz paula andrea,femenino,33,ind,independientes


In [312]:
partidos = generate_dictionaries(new_2021["ID Partido"], new_2021["Partido"])
id_partidos = generate_dictionaries(new_2021["Partido"], new_2021["ID Partido"])
regiones = generate_dictionaries(new_2021["Region"], new_2021["ID Region"])

In [313]:
partidos_faltantes = {'partido progresista':'pp', 'democracia regional patagonica':'drp',
             'partido izquierda ciudadana de chile': 'pic', 'partido igualdad': 'igualdad', 
             'partido radical socialdemocrata': 'prs', 'todos': 'todos', 
             'partido de trabajadores revolucionarios': 'pti', 
             'partido evolucion politica': 'evopoli', 'amplitud': 'amplitud', 
             'partido regionalista independiente': 'pri', 
             'partido mas region': 'pmr', 'partido pais': 'pp', 'poder': 'poder',
             'partido ciudadanos': 'ciudadanos'}
partidos.update(partidos_faltantes)

In [314]:
regiones[0.0] = "nacional"

In [315]:
store_df(new_2021, "clean/Candidaturas_2021.csv")

### Candidaturas 2013-2017

In [316]:
df_13_17 = pd.read_csv("Candidaturas_2013_2017.csv", sep=";")
df_13_17.head()

,Ano,ID Eleccion,ID Region,Territorio,Partido,Candidato,Sexo,Edad
0,2013,PRESIDENCIAL,NaN,NACIONAL,PARTIDO ECOLOGISTA VERDE,SFEIR YOUNIS ALFREDO,HOMBRE,66
1,2013,PRESIDENCIAL,NaN,NACIONAL,PARTIDO SOCIALISTA DE CHILE,BACHELET JERIA MICHELLE,MUJER,62
2,2013,PRESIDENCIAL,NaN,NACIONAL,PARTIDO REGIONALISTA DE LOS INDEPENDIENTES,ISRAEL ZIPPER RICARDO,HOMBRE,63
3,2013,PRESIDENCIAL,NaN,NACIONAL,INDEPENDIENTE,PARISI FERNANDEZ FRANCO ALDO,HOMBRE,46
4,2013,PRESIDENCIAL,NaN,NACIONAL,INDEPENDIENTE,JOCELYN-HOLT LETELIER TOMAS,HOMBRE,50


In [317]:
# el id de las presidenciales será 0.0
df_13_17["ID Region"] = df_13_17["ID Region"].replace(np.nan, "0.0")

# se hacen nulos los datos no enteros
df_13_17["Edad"] = df_13_17["Edad"].replace(r'[^1-9]+', np.nan, regex=True)

In [318]:
# ID de tabla se genera al poblar DB
new_13_17 = pd.DataFrame(columns={
    'Ano': pd.Series(dtype=int),
    'ID Eleccion': pd.Series(dtype=str),
    'ID Region': pd.Series(dtype=float),
    'Region': pd.Series(dtype=str),
    'Territorio': pd.Series(dtype=str),
    'Candidato': pd.Series(dtype=str),
    'Sexo': pd.Series(dtype=str),
    'Edad': pd.Series(dtype=int),
    'ID Partido': pd.Series(dtype=str),
    'Partido': pd.Series(dtype=str)
})

def fill_candidaturas(new, old):
    new['Ano'] = old['Ano'].astype(int)
    new['ID Eleccion'] = old['ID Eleccion'].astype(str).apply(modify_string)
    new['ID Region'] = old['ID Region'].astype(float)
    new['Territorio'] = old['Territorio'].astype(str).apply(modify_string)
    new['Candidato'] = old['Candidato'].astype(str).apply(modify_string)
    new['Sexo'] = old['Sexo'].astype(str).apply(modify_string)
    new['Edad'] = old['Edad'].astype(int).apply(modify_int)
    new['Partido'] = old['Partido'].astype(str).apply(modify_string)
    # elimino duplicados
    new = new.drop_duplicates()
    return new


df_13_17.dropna(inplace=True)
new_13_17 = fill_candidaturas(new_13_17, df_13_17)

new_13_17.head()

,Ano,ID Eleccion,ID Region,Region,Territorio,Candidato,Sexo,Edad,ID Partido,Partido
0,2013,presidencial,0.0,NaN,nacional,sfeir younis alfredo,hombre,66,NaN,partido ecologista verde
1,2013,presidencial,0.0,NaN,nacional,bachelet jeria michelle,mujer,62,NaN,partido socialista de chile
2,2013,presidencial,0.0,NaN,nacional,israel zipper ricardo,hombre,63,NaN,partido regionalista de los independientes
3,2013,presidencial,0.0,NaN,nacional,parisi fernandez franco aldo,hombre,46,NaN,independiente
6,2013,presidencial,0.0,NaN,nacional,claude reyes marcel,hombre,56,NaN,partido humanista


In [319]:
new_13_17["Region"] = new_13_17["ID Region"].map(regiones)
new_13_17["ID Partido"] = new_13_17["Partido"].map(partidos)

In [320]:
new_13_17.head()

,Ano,ID Eleccion,ID Region,Region,Territorio,Candidato,Sexo,Edad,ID Partido,Partido
0,2013,presidencial,0.0,nacional,nacional,sfeir younis alfredo,hombre,66,pev,partido ecologista verde
1,2013,presidencial,0.0,nacional,nacional,bachelet jeria michelle,mujer,62,ps,partido socialista de chile
2,2013,presidencial,0.0,nacional,nacional,israel zipper ricardo,hombre,63,NaN,partido regionalista de los independientes
3,2013,presidencial,0.0,nacional,nacional,parisi fernandez franco aldo,hombre,46,ind,independiente
6,2013,presidencial,0.0,nacional,nacional,claude reyes marcel,hombre,56,ph,partido humanista


In [321]:
new_13_17.dropna(inplace=True)
new_13_17.head()

,Ano,ID Eleccion,ID Region,Region,Territorio,Candidato,Sexo,Edad,ID Partido,Partido
0,2013,presidencial,0.0,nacional,nacional,sfeir younis alfredo,hombre,66,pev,partido ecologista verde
1,2013,presidencial,0.0,nacional,nacional,bachelet jeria michelle,mujer,62,ps,partido socialista de chile
3,2013,presidencial,0.0,nacional,nacional,parisi fernandez franco aldo,hombre,46,ind,independiente
6,2013,presidencial,0.0,nacional,nacional,claude reyes marcel,hombre,56,ph,partido humanista
7,2013,presidencial,0.0,nacional,nacional,miranda meneses roxana,mujer,46,igualdad,partido igualdad


In [322]:
store_df(new_13_17, "clean/Candidaturas_2013_2017.csv")

## Archivo fusión candidaturas

In [323]:
candidaturas = pd.concat([new_13_17, new_2021], axis=0)
candidaturas.index.name = 'ID'
candidaturas.head()

,Ano,ID Eleccion,ID Region,Region,Territorio,Candidato,Sexo,Edad,ID Partido,Partido
ID,,,,,,,,,,
0,2013,presidencial,0.0,nacional,nacional,sfeir younis alfredo,hombre,66,pev,partido ecologista verde
1,2013,presidencial,0.0,nacional,nacional,bachelet jeria michelle,mujer,62,ps,partido socialista de chile
3,2013,presidencial,0.0,nacional,nacional,parisi fernandez franco aldo,hombre,46,ind,independiente
6,2013,presidencial,0.0,nacional,nacional,claude reyes marcel,hombre,56,ph,partido humanista
7,2013,presidencial,0.0,nacional,nacional,miranda meneses roxana,mujer,46,igualdad,partido igualdad


In [324]:
store_df(candidaturas, "clean/Candidaturas_2013_2017_2021.csv")

### Total afiliados partidos

In [325]:
afiliados = pd.read_csv("Total_afiliados_partidos.csv", sep=";")
afiliados.head()

,Partido,Categoria,Comuna,Rango edad,Region,Sexo
0,EVOLUCION POLITICA,Nuevo,Lo Espejo,20-24 años,Metropolitana De Santiago,Masculino
1,EVOLUCION POLITICA,Nuevo,NaN,50-54 años,INTERNACIONAL,Femenino
2,EVOLUCION POLITICA,Nuevo,Vitacura,60-64 años,Metropolitana De Santiago,Masculino
3,EVOLUCION POLITICA,Nuevo,Arica,65-69 años,Arica Y Parinacota,Masculino
4,EVOLUCION POLITICA,Nuevo,Arica,25-29 años,Arica Y Parinacota,Femenino


In [326]:
# ID de tabla se genera al poblar DB
new_afiliados = pd.DataFrame(columns={
    'ID Partido': pd.Series(dtype=str),
    'Partido': pd.Series(dtype=str),
    'Categoria': pd.Series(dtype=str),
    'Comuna': pd.Series(dtype=str),
    'Rango edad': pd.Series(dtype=str),
    'Region': pd.Series(dtype=str),
    'Sexo': pd.Series(dtype=str)
})

def fill_afiliados(new, old):
    new['Partido'] = old['Partido'].astype(str).apply(modify_string)
    new['Categoria'] = old['Categoria'].astype(str).apply(modify_string)
    new['Comuna'] = old['Comuna'].astype(str).apply(modify_string)
    new['Rango edad'] = old['Rango edad'].astype(str).apply(modify_string)
    new['Region'] = old['Region'].astype(str).apply(modify_string)
    new['Sexo'] = old['Sexo'].astype(str).apply(modify_string)
    # elimino duplicados
    new = new.drop_duplicates()
    return new

# se eliminan datos nulos de entrada
afiliados.dropna(inplace=True)
new_afiliados = fill_afiliados(new_afiliados, afiliados)
new_afiliados["ID Partido"] = new_afiliados["Partido"].map(partidos)
new_afiliados.head()

,ID Partido,Partido,Categoria,Comuna,Rango edad,Region,Sexo
0,evopoli,evolucion politica,nuevo,lo espejo,20-24 anos,metropolitana de santiago,masculino
2,evopoli,evolucion politica,nuevo,vitacura,60-64 anos,metropolitana de santiago,masculino
3,evopoli,evolucion politica,nuevo,arica,65-69 anos,arica y parinacota,masculino
4,evopoli,evolucion politica,nuevo,arica,25-29 anos,arica y parinacota,femenino
5,evopoli,evolucion politica,nuevo,arica,40-44 anos,arica y parinacota,masculino


In [327]:
# creacion archivo
new_afiliados.index.name = 'ID'
store_df(new_afiliados, "clean/Total_afiliados_partidos.csv")

### Ingresos Gastos 2017 Candidatos

In [328]:
df_gastos_17 =pd.read_csv("Ingresos_Gastos_2017_Candidatos.csv", sep=";")
df_gastos_17.head()

,Tipo Planilla,ID Eleccion,ID Region,Nombre Candidato,Afiliacion,Nombre Partido,Aportante/Proveedor,DV,Nombre Aportante/Proveedor,Fecha Documento,Tipo Documento,Descripción Tipo Documento,Tipo Cuenta,Descripción Tipo Cuenta,ID Documento,Glosa Documento,Monto,Unnamed: 17,Unnamed: 18
0,INGRESOS,PRESIDENTE,8,ALEJANDRO NAVARRO BRAIN,Afiliado,PARTIDO PAIS,8510034.0,3,ALEJANDRO NAVARRO BRAIN,06/10/17,OI,Otros Documentos de Ingresos,110,Aporte personal del candidato,7734.0,PROPIO,20000000,NaN,NaN
1,INGRESOS,PRESIDENTE,8,ALEJANDRO NAVARRO BRAIN,Afiliado,PARTIDO PAIS,8510034.0,3,ALEJANDRO NAVARRO BRAIN,06/11/17,OI,Otros Documentos de Ingresos,110,Aporte personal del candidato,18205.0,PROPIO,6000000,NaN,NaN
2,INGRESOS,PRESIDENTE,8,ALEJANDRO NAVARRO BRAIN,Afiliado,PARTIDO PAIS,8510034.0,3,ALEJANDRO NAVARRO BRAIN,09/11/17,OI,Otros Documentos de Ingresos,110,Aporte personal del candidato,19707.0,PROPIO,20000000,NaN,NaN
3,INGRESOS,PRESIDENTE,8,ALEJANDRO NAVARRO BRAIN,Afiliado,PARTIDO PAIS,15216914.0,0,MARCELO IVAN CARDENAS ALVAREZ,14/11/17,OI,Otros Documentos de Ingresos,135,Aportes de personas naturales,20723.0,PÚBLICO,6000000,NaN,NaN
4,INGRESOS,PRESIDENTE,8,ALEJANDRO NAVARRO BRAIN,Afiliado,PARTIDO PAIS,8510034.0,3,ALEJANDRO NAVARRO BRAIN,13/11/17,OI,Otros Documentos de Ingresos,110,Aporte personal del candidato,20740.0,PROPIO,10000000,NaN,NaN


In [329]:
df_gastos_17 = df_gastos_17.rename(columns=lambda x: x.strip())

In [330]:
# si el dv es k se pasa a 10.0
df_gastos_17["DV"] = df_gastos_17["DV"].replace('k', 10.0)

In [331]:
# ID de tabla se genera al poblar DB
new_gastos_17 = pd.DataFrame(columns={
    'Tipo Planilla': pd.Series(dtype=str),
    'ID Eleccion': pd.Series(dtype=str),
    'ID Region': pd.Series(dtype=int),
    'Nombre Candidato': pd.Series(dtype=str),
    'Afiliacion': pd.Series(dtype=str),
    'ID Partido': pd.Series(dtype=str),
    'Nombre Partido': pd.Series(dtype=str),
    'Aportante/Proveedor': pd.Series(dtype=float),
    'DV': pd.Series(dtype=float),
    'Nombre Aportante/Proveedor': pd.Series(dtype=str),
    'Fecha': pd.Series(dtype=str),
#     'Documento': pd.Series(dtype=str),
    'Tipo Documento': pd.Series(dtype=str),
    'Descripcion Tipo Documento': pd.Series(dtype=str),
    'Tipo Cuenta': pd.Series(dtype=int),
    'Descripcion Tipo Cuenta': pd.Series(dtype=str),
    'ID Documento': pd.Series(dtype=float),
    'Glosa Documento': pd.Series(dtype=str),
    'Monto': pd.Series(dtype=int)
})

def fill_gastos(new, old):
    new['Tipo Planilla'] = old['Tipo Planilla'].astype(str).apply(modify_string)
    new['ID Eleccion'] = old['ID Eleccion'].astype(str).apply(modify_string)
    new['ID Region'] = old['ID Region'].astype(int)
    new['Nombre Candidato'] = old['Nombre Candidato'].astype(str).apply(modify_string)
    new['Afiliacion'] = old['Afiliacion'].astype(str).apply(modify_string)
    new['Nombre Partido'] = old['Nombre Partido'].astype(str).apply(modify_string)
    new['Aportante/Proveedor'] = old['Aportante/Proveedor'].astype(float)
    new['DV'] = old['DV'].astype(float)
    new['Nombre Aportante/Proveedor'] = old['Nombre Aportante/Proveedor'].astype(str).apply(modify_string)
    new['Fecha'] = old['Fecha Documento'].astype(str).apply(modify_string)
    new['Tipo Documento'] = old['Tipo Documento'].astype(str).apply(modify_string)
    new['Descripcion Tipo Documento'] = old['Descripción Tipo Documento'].astype(str).apply(modify_string)
    new['Tipo Cuenta'] = old['Tipo Cuenta'].astype(int)
    new['Descripcion Tipo Cuenta'] = old['Descripción Tipo Cuenta'].astype(str).apply(modify_string)
    new['ID Documento'] = old['ID Documento'].astype(float)
    new['Glosa Documento'] = old['Glosa Documento'].astype(str).apply(modify_string)
    new['Monto'] = old['Monto'].astype(int)
    # elimino duplicados
    new = new.drop_duplicates()
    return new


new_gastos_17 = fill_gastos(new_gastos_17, df_gastos_17)

new_gastos_17.head()

,Tipo Planilla,ID Eleccion,ID Region,Nombre Candidato,Afiliacion,ID Partido,Nombre Partido,Aportante/Proveedor,DV,Nombre Aportante/Proveedor,Fecha,Tipo Documento,Descripcion Tipo Documento,Tipo Cuenta,Descripcion Tipo Cuenta,ID Documento,Glosa Documento,Monto
0,ingresos,presidente,8,alejandro navarro brain,afiliado,NaN,partido pais,8510034.0,3.0,alejandro navarro brain,06/10/17,oi,otros documentos de ingresos,110,aporte personal del candidato,7734.0,propio,20000000
1,ingresos,presidente,8,alejandro navarro brain,afiliado,NaN,partido pais,8510034.0,3.0,alejandro navarro brain,06/11/17,oi,otros documentos de ingresos,110,aporte personal del candidato,18205.0,propio,6000000
2,ingresos,presidente,8,alejandro navarro brain,afiliado,NaN,partido pais,8510034.0,3.0,alejandro navarro brain,09/11/17,oi,otros documentos de ingresos,110,aporte personal del candidato,19707.0,propio,20000000
3,ingresos,presidente,8,alejandro navarro brain,afiliado,NaN,partido pais,15216914.0,0.0,marcelo ivan cardenas alvarez,14/11/17,oi,otros documentos de ingresos,135,aportes de personas naturales,20723.0,publico,6000000
4,ingresos,presidente,8,alejandro navarro brain,afiliado,NaN,partido pais,8510034.0,3.0,alejandro navarro brain,13/11/17,oi,otros documentos de ingresos,110,aporte personal del candidato,20740.0,propio,10000000


In [332]:
new_gastos_17["ID Partido"] = new_gastos_17["Nombre Partido"].map(partidos)

In [333]:
new_gastos_17.dropna(inplace = True)
new_gastos_17.head()

,Tipo Planilla,ID Eleccion,ID Region,Nombre Candidato,Afiliacion,ID Partido,Nombre Partido,Aportante/Proveedor,DV,Nombre Aportante/Proveedor,Fecha,Tipo Documento,Descripcion Tipo Documento,Tipo Cuenta,Descripcion Tipo Cuenta,ID Documento,Glosa Documento,Monto
0,ingresos,presidente,8,alejandro navarro brain,afiliado,pp,partido pais,8510034.0,3.0,alejandro navarro brain,06/10/17,oi,otros documentos de ingresos,110,aporte personal del candidato,7734.0,propio,20000000
1,ingresos,presidente,8,alejandro navarro brain,afiliado,pp,partido pais,8510034.0,3.0,alejandro navarro brain,06/11/17,oi,otros documentos de ingresos,110,aporte personal del candidato,18205.0,propio,6000000
2,ingresos,presidente,8,alejandro navarro brain,afiliado,pp,partido pais,8510034.0,3.0,alejandro navarro brain,09/11/17,oi,otros documentos de ingresos,110,aporte personal del candidato,19707.0,propio,20000000
3,ingresos,presidente,8,alejandro navarro brain,afiliado,pp,partido pais,15216914.0,0.0,marcelo ivan cardenas alvarez,14/11/17,oi,otros documentos de ingresos,135,aportes de personas naturales,20723.0,publico,6000000
4,ingresos,presidente,8,alejandro navarro brain,afiliado,pp,partido pais,8510034.0,3.0,alejandro navarro brain,13/11/17,oi,otros documentos de ingresos,110,aporte personal del candidato,20740.0,propio,10000000


In [336]:
new_gastos_17.index.name = 'ID'

In [337]:
store_df(new_gastos_17, "clean/Ingresos_Gastos_2017_Candidatos.csv")

### Aportes públicos anual

In [338]:
df_publicos =pd.read_csv("Aportes_publicos_anual.csv", sep=";")
df_publicos.head()

,Nombre Partido,Monto,Fecha de Pago,Trimestre,Año
0,RENOVACION NACIONAL,248112449.0,09-05-2016,PRIMER APORTE,2016.0
1,DEMOCRATA CRISTIANO,273623453.0,24-06-2016,PRIMER APORTE,2016.0
2,UNION DEMOCRATA INDEPENDIENTE,324176809.0,19-05-2016,PRIMER APORTE,2016.0
3,POR LA DEMOCRACIA,202470445.0,09-05-2016,PRIMER APORTE,2016.0
4,SOCIALISTA DE CHILE,203965625.0,09-05-2016,PRIMER APORTE,2016.0


In [339]:
df_publicos.dropna(inplace=True)
df_publicos = df_publicos.rename(columns=lambda x: x.strip())

In [340]:
df_publicos["Fecha de Pago"] = df_publicos["Fecha de Pago"].replace('Sin derecho a pago.', np.nan)

In [341]:
# ID de tabla se genera al poblar DB
new_publicos = pd.DataFrame(columns={
    'ID Partido': pd.Series(dtype=str),
    'Nombre Partido': pd.Series(dtype=str),
    'Monto': pd.Series(dtype=int),
    'Fecha de Pago': pd.Series(dtype=float),
    'Trimestre': pd.Series(dtype=str),
    'Ano': pd.Series(dtype=int)
})

def fill_publicos(new, old):
    new['Nombre Partido'] = old['Nombre Partido'].astype(str).apply(modify_string)
    new['Monto'] = old['Monto'].astype(int)
    new['Fecha de Pago'] = pd.to_datetime(old['Fecha de Pago'])
    new['Trimestre'] = old['Trimestre'].astype(str).apply(modify_string)
    new['Ano'] = old['Año'].astype(int)
    # elimino duplicados
    new = new.drop_duplicates()
    return new


df_publicos.dropna(inplace=True)
new_publicos = fill_publicos(new_publicos, df_publicos)

new_publicos.head()

,ID Partido,Nombre Partido,Monto,Fecha de Pago,Trimestre,Ano
0,NaN,renovacion nacional,248112449,2016-09-05,primer aporte,2016
1,NaN,democrata cristiano,273623453,2016-06-24,primer aporte,2016
2,NaN,union democrata independiente,324176809,2016-05-19,primer aporte,2016
3,NaN,por la democracia,202470445,2016-09-05,primer aporte,2016
4,NaN,socialista de chile,203965625,2016-09-05,primer aporte,2016


In [344]:
new_publicos["ID Partido"] = new_publicos["Nombre Partido"].map(partidos)

In [345]:
new_publicos.head()

,ID Partido,Nombre Partido,Monto,Fecha de Pago,Trimestre,Ano
0,rn,renovacion nacional,248112449,2016-09-05,primer aporte,2016
1,NaN,democrata cristiano,273623453,2016-06-24,primer aporte,2016
2,udi,union democrata independiente,324176809,2016-05-19,primer aporte,2016
3,NaN,por la democracia,202470445,2016-09-05,primer aporte,2016
4,NaN,socialista de chile,203965625,2016-09-05,primer aporte,2016


In [355]:
nombres = []
for a,b in zip(new_publicos["ID Partido"], new_publicos["Nombre Partido"]):
    if pd.isnull(a):
        b = "partido " + b
    nombres.append(b)
new_publicos["Nombre Partido"] = pd.Series(nombres)
new_publicos.head()

,ID Partido,Nombre Partido,Monto,Fecha de Pago,Trimestre,Ano
0,rn,renovacion nacional,248112449,2016-09-05,primer aporte,2016
1,NaN,partido democrata cristiano,273623453,2016-06-24,primer aporte,2016
2,udi,union democrata independiente,324176809,2016-05-19,primer aporte,2016
3,NaN,partido por la democracia,202470445,2016-09-05,primer aporte,2016
4,NaN,partido socialista de chile,203965625,2016-09-05,primer aporte,2016


In [356]:
new_publicos["ID Partido"] = new_publicos["Nombre Partido"].map(partidos)

In [358]:
new_publicos.dropna(inplace=True)
new_publicos.head()

,ID Partido,Nombre Partido,Monto,Fecha de Pago,Trimestre,Ano
0,rn,renovacion nacional,248112449,2016-09-05,primer aporte,2016
1,pdc,partido democrata cristiano,273623453,2016-06-24,primer aporte,2016
2,udi,union democrata independiente,324176809,2016-05-19,primer aporte,2016
3,ppd,partido por la democracia,202470445,2016-09-05,primer aporte,2016
4,ps,partido socialista de chile,203965625,2016-09-05,primer aporte,2016


In [359]:
new_publicos.index.name = 'ID'

In [360]:
store_df(new_publicos, "clean/Aportes_publicos_anual.csv")

### Aportes privados 2017